<a href="https://colab.research.google.com/github/EduardoMoraesRitter/introducao-a-sistemas-de-recomendacao-com-python/blob/master/KNN_Gerando_recomenda%C3%A7%C3%B5es_baseado_em_um_usu%C3%A1rio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

filmes = pd.read_csv('movies.csv')
notas = pd.read_csv('ratings.csv')

filmes.columns = ['filmeId', 'titulo', 'generos']
filmes = filmes.set_index('filmeId')
notas.columns = ['usuarioId', 'filmeId', 'nota', 'momento']

#total de votos
filmes['total_de_votos'] = notas['filmeId'].value_counts()

#media de nota por filme
filmes['notas_medias'] = notas.groupby('filmeId').mean()['nota']

In [117]:
#filtar um usuario, pegar duas coluna, colocar com indice
notas_usuario = notas.query('usuarioId==1')
notas_usuario = notas_usuario[['filmeId', 'nota']].set_index("filmeId")
notas_usuario.head()

,nota
filmeId,
1,4.0
3,4.0
6,4.0
47,5.0
50,5.0


#Parametros para teste

In [0]:
def notas_usuario(usuarioId):
    notas_usuario = notas.query('usuarioId==%d' % usuarioId)
    notas_usuario = notas_usuario[['filmeId', 'nota']].set_index("filmeId")
    return notas_usuario

In [0]:
def distancia_usuarios(uId1,uId2, minimo=5):
  u1N = notas_usuario(uId1)
  u2N = notas_usuario(uId2)
  diferencas = u1N.join(u2N, lsuffix="u_esq", rsuffix="u_dir").dropna()
  if(len(diferencas) < minimo):
    return None
  distancia_usu1_usu2 = distancia_vetor(diferencas['notau_esq'], diferencas['notau_dir'])
  return [uId1, uId2, distancia_usu1_usu2]

In [0]:
def distancia_todos(voce, numero_usuario_analizar = None):
  todos_usuarios = notas['usuarioId'].unique()
  if numero_usuario_analizar:
     todos_usuarios = todos_usuarios[:numero_usuario_analizar]
  distancias = [distancia_usuarios(voce,usuario_id) for usuario_id in todos_usuarios]
  distancias = list(filter(None, distancias))
  distancias = pd.DataFrame(distancias, columns = ["voce", "outro", "distancia"])
  return distancias

In [121]:
def mais_proxima(voce_id, numero_usuario_analizar = None):
  distancias = distancia_todos(voce_id, numero_usuario_analizar=numero_usuario_analizar)
  distancias = distancias.sort_values("distancia").set_index("outro").drop(voce_id)
  return distancias
mais_proxima(1, numero_usuario_analizar=5)

,voce,distancia
outro,,
5,1,3.741657
3,1,8.200610
4,1,11.135529


#pegar o usuario mais similar, pegar os filmes que ele assistiu e vc nao 

In [122]:
voce = 1
notas_voce = notas_usuario(voce)
filmes_voce = notas_voce.index
similares = mais_proxima(voce, numero_usuario_analizar=5)
#pega a linha iloc, pega o name
similar = similares.iloc[0].name
notas_similar = notas_usuario(similar)
#remover os filmes q vc assistiu da lista, que vc viu q ele nao viu
notas_similar = notas_similar.drop(filmes_voce, errors='ignore')
notas_similar = notas_similar.sort_values("nota", ascending=False)
notas_similar.head()

,nota
filmeId,
475,5.0
595,5.0
58,5.0
594,5.0
290,5.0


In [0]:
def sugere_para(voce, numero_usuario_analizar=None):
  notas_voce = notas_usuario(voce)
  filmes_voce = notas_voce.index
  similares = mais_proxima(voce, numero_usuario_analizar=numero_usuario_analizar)
  similar = similares.iloc[0].name
  notas_similar = notas_usuario(similar)
  notas_similar = notas_similar.drop(filmes_voce, errors='ignore')
  recomendacoes = notas_similar.sort_values("nota", ascending=False)
  return recomendacoes.join(filmes)

In [124]:
sugere_para(1).head()

,nota,titulo,generos,total_de_votos,notas_medias
filmeId,,,,,
8636,5.0,Spider-Man 2 (2004),Action|Adventure|Sci-Fi|IMAX,79.0,3.803797
58559,5.0,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX,149.0,4.238255
33794,5.0,Batman Begins (2005),Action|Crime|IMAX,116.0,3.862069
4993,5.0,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy,198.0,4.106061
5349,5.0,Spider-Man (2002),Action|Adventure|Sci-Fi|Thriller,122.0,3.540984


#sugerir baseado em varios usuarios

In [0]:
def mais_proxima(voce_id, n_mais_proximos=10, numero_usuario_analizar = None):
  distancias = distancia_todos(voce_id, numero_usuario_analizar=numero_usuario_analizar)
  distancias = distancias.sort_values("distancia").set_index("outro").drop(voce_id)
  return distancias.head(n_mais_proximos)

In [126]:
mais_proxima(1, 3, 200)

,voce,distancia
outro,,
77,1,0.0
49,1,1.0
9,1,1.0


In [128]:
voce = 1
n_mais_proximos = 10
numero_usuario_analizar = 200

notas_voce = notas_usuario(voce)
filmes_voce = notas_voce.index

similares = mais_proxima(voce, n_mais_proximos=n_mais_proximos, numero_usuario_analizar=numero_usuario_analizar)
usuarios_similares = similares.index
#pega a nota de todos esse usuario
#notas.query('usuarios in [4,6]') muito ruim
#nao tem problema do index esta repetido
notas_similares = notas.set_index("usuarioId").loc[usuarios_similares]

#fazer uma media dentre as notas, retorna um serie
notas_similares.groupby("filmeId").mean()["nota"]

#retona com [] transforma para dataframe
recomendacoes = notas_similares.groupby("filmeId").mean()[["nota"]]
recomendacoes = recomendacoes.sort_values("nota", ascending=False)

#junta com os filmes
recomendacoes.join(filmes).head()

#antes
# similar = similares.iloc[0].name
# notas_similar = notas_usuario(similar)
# notas_similar = notas_similar.drop(filmes_voce, errors='ignore')
# notas_similar = notas_similar.sort_values("nota", ascending=False)
# notas_similar.head()

,nota,titulo,generos,total_de_votos,notas_medias
filmeId,,,,,
187593,5.0,Deadpool 2 (2018),Action|Comedy|Sci-Fi,12.0,3.875000
3996,5.0,"Crouching Tiger, Hidden Dragon (Wo hu cang lon...",Action|Drama|Romance,110.0,3.836364
5349,5.0,Spider-Man (2002),Action|Adventure|Sci-Fi|Thriller,122.0,3.540984
613,5.0,Jane Eyre (1996),Drama|Romance,8.0,3.187500
608,5.0,Fargo (1996),Comedy|Crime|Drama|Thriller,181.0,4.116022


In [0]:
def knn(voce, k_mais_proximos = 10, numero_usuario_analizar = None):
  notas_voce = notas_usuario(voce)
  filmes_voce = notas_voce.index

  similares = mais_proxima(voce, n_mais_proximos=k_mais_proximos, numero_usuario_analizar=numero_usuario_analizar)
  usuarios_similares = similares.index
  notas_similares = notas.set_index("usuarioId").loc[usuarios_similares]
  recomendacoes = notas_similares.groupby("filmeId").mean()[["nota"]]
  recomendacoes = recomendacoes.sort_values("nota", ascending=False)
  return recomendacoes.join(filmes)

In [135]:
knn(1, 10, 50).head()

,nota,titulo,generos,total_de_votos,notas_medias
filmeId,,,,,
187593,5.0,Deadpool 2 (2018),Action|Comedy|Sci-Fi,12.0,3.875000
68157,5.0,Inglourious Basterds (2009),Action|Drama|War,88.0,4.136364
475,5.0,In the Name of the Father (1993),Drama,25.0,4.300000
33794,5.0,Batman Begins (2005),Action|Crime|IMAX,116.0,3.862069
7153,5.0,"Lord of the Rings: The Return of the King, The...",Action|Adventure|Drama|Fantasy,185.0,4.118919


In [136]:
knn(1).head()

,nota,titulo,generos,total_de_votos,notas_medias
filmeId,,,,,
1704,5.0,Good Will Hunting (1997),Drama|Romance,141.0,4.078014
57504,5.0,"Girl Who Leapt Through Time, The (Toki o kaker...",Animation|Comedy|Drama|Romance|Sci-Fi,10.0,4.100000
38304,5.0,No Direction Home: Bob Dylan (2005),Documentary,4.0,4.625000
48394,5.0,"Pan's Labyrinth (Laberinto del fauno, El) (2006)",Drama|Fantasy|Thriller,81.0,3.814815
2300,5.0,"Producers, The (1968)",Comedy,33.0,3.969697


In [137]:
knn(1).tail()

,nota,titulo,generos,total_de_votos,notas_medias
filmeId,,,,,
5507,1.0,xXx (2002),Action|Crime|Thriller,24.0,2.770833
4131,1.0,Making Mr. Right (1987),Comedy|Romance|Sci-Fi,3.0,1.833333
5891,1.0,I Spit on Your Grave (Day of the Woman) (1978),Horror|Thriller,4.0,2.625000
5962,1.0,Body of Evidence (1993),Drama|Thriller,1.0,1.000000
87232,0.5,X-Men: First Class (2011),Action|Adventure|Sci-Fi|Thriller|War,43.0,3.790698


#Notas de filmes com mais de 50 notas


In [140]:
filmes_mais_50_votos = filmes.query("total_de_votos >= 50")
filmes_mais_50_votos.index
#colocar as nota para index ser o filme, localiza so os mais de 50 votos
notas = notas.set_index("filmeId").loc[filmes_mais_50_votos.index]

notas.head()

,usuarioId,nota,momento
filmeId,,,
1,1,4.0,964982703
1,5,4.0,847434962
1,7,4.5,1106635946
1,15,2.5,1510577970
1,17,4.5,1305696483


In [141]:
#precisa voltar o index original, para qunado tentar acessa a coluna filmeID nao da
notas = notas.reset_index()
notas.head()

,filmeId,usuarioId,nota,momento
0,1,1,4.0,964982703
1,1,5,4.0,847434962
2,1,7,4.5,1106635946
3,1,15,2.5,1510577970
4,1,17,4.5,1305696483


In [142]:
knn(1).head()

,nota,titulo,generos,total_de_votos,notas_medias
filmeId,,,,,
56174,5.0,I Am Legend (2007),Action|Horror|Sci-Fi|Thriller|IMAX,62.0,3.483871
48394,5.0,"Pan's Labyrinth (Laberinto del fauno, El) (2006)",Drama|Fantasy|Thriller,81.0,3.814815
1196,5.0,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Sci-Fi,211.0,4.215640
1197,5.0,"Princess Bride, The (1987)",Action|Adventure|Comedy|Fantasy|Romance,142.0,4.232394
1198,5.0,Raiders of the Lost Ark (Indiana Jones and the...,Action|Adventure,200.0,4.207500


In [0]:
#cuidado com o drop pq ele pode nao estar la usar o errors='ignore'
def mais_proxima(voce_id, n_mais_proximos=10, numero_usuario_analizar = None):
  distancias = distancia_todos(voce_id, numero_usuario_analizar=numero_usuario_analizar)
  distancias = distancias.sort_values("distancia").set_index("outro").drop(voce_id, errors='ignore')
  return distancias.head(n_mais_proximos)

In [0]:
#tirar os filmes que tem pouco votos dentro da pessoas que sao semelhantes a mim
def knn(voce_id, k_mais_proximos = 10, numero_usuario_analizar = None):
  notas_voce = notas_usuario(voce_id)
  filmes_voce = notas_voce.index

  similares = mais_proxima(voce_id, n_mais_proximos=k_mais_proximos, numero_usuario_analizar=numero_usuario_analizar)
  usuarios_similares = similares.index
  notas_similares = notas.set_index("usuarioId").loc[usuarios_similares]
  recomendacoes = notas_similares.groupby("filmeId").mean()[["nota"]]
  
  aparicoes = notas_similares.groupby("filmeId").count()[['nota']]
  filtro_minimo = k_mais_proximos / 2
  recomendacoes = recomendacoes.join(aparicoes, lsuffix='_media_usuarios', rsuffix='_qtd_votos_usuarios')
  recomendacoes = recomendacoes.query("nota_qtd_votos_usuarios >= %.2f" % filtro_minimo)
  recomendacoes = recomendacoes.sort_values("nota_media_usuarios", ascending=False)

  recomendacoes = recomendacoes.drop(filmes_voce, errors='ignore')

  return recomendacoes.join(filmes)

In [182]:
knn(1).head()

,nota_media_usuarios,nota_qtd_votos_usuarios,titulo,generos,total_de_votos,notas_medias
filmeId,,,,,,
7153,4.750000,6,"Lord of the Rings: The Return of the King, The...",Action|Adventure|Drama|Fantasy,185.0,4.118919
58559,4.750000,6,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX,149.0,4.238255
4993,4.714286,7,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy,198.0,4.106061
318,4.583333,6,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
109487,4.583333,6,Interstellar (2014),Sci-Fi|IMAX,73.0,3.993151
